In [10]:
import numpy as np
import scipy as sp
import math
# Define known parameters
rho1 = 1573.2; mu1 = 65e-3; N1 = 18000; D1 = 51e-3; M1 = 0.85; P1 = 10.4e5
#rho2 = 1000; mu2 = 10e-3
N2 = 2900; D2 = 80e-3
g = 9.807  # Acceleration due to gravity in m/s^2

Q1 = M1/rho1

# Define lower and upper bounds
lb = [1e-3, 500, 0.1, 0.5e5]  # Lower bounds
ub = [100e-3, 1500, 1.0, 4e5]  # Upper bounds
bounds = [(low, high) for low, high in zip(lb, ub)]
#np.random.seed(100)

# Define the objective function (sum of squared residuals)
def pump_equations(x):
    mu2, rho2, M2, P2 = x
    
    # Specific speed equation residual
    F1 = (N1 * np.sqrt(M1/rho1) / (g * P1/(rho1*g))**(3/4)) - (N2 * np.sqrt(M2/rho2) / (g * P2/(rho2*g))**(3/4))
    
    # Flow coefficient equation residual
    F2 = (M1/rho1 / (N1 * D1**3)) - (M2/rho2 / (N2 * D2**3))
    
    # Head coefficient equation residual
    F3 = (g * P1/(rho1*g) / (N1**2 * D1**2)) - (g * P2/(rho2*g) / (N2**2 * D2**2))
    
    # Reynolds number equation residual
    F4 = (rho1 * N1 * D1**2 / mu1) - (rho2 * N2 * D2**2 / mu2)
    
    # Sum of squared residuals
    #return F1**2 + F2**2 + F3**2 + F4**2
    return [F1, F2, F3, F4]

# Normalization function for better scaling
def normalize_residuals(residuals):
    # Scale each residual by a factor that brings them into a similar range
    scales = [1.0, 1.0e-3, 1.0e-3, 1]  # Example scales, adjust as needed
    return [residuals[i] / scales[i] for i in range(len(residuals))]

# Objective function for minimization (sum of squares of normalized residuals)
def objective_function(x):
    residuals = pump_equations(x)
    normalized_residuals = normalize_residuals(residuals)
    return sum([r**2 for r in normalized_residuals])


# Initial guesses for unknowns
x0 = np.array([30e-3, 1000, 0.4, 2e5])

# Perform optimization using 'SLSQP' (Sequential Least Squares Programming)
#result = sp.optimize.minimize(objective_function, x0, bounds=bounds, method='trust-constr', options={'disp': True, 'maxiter': 10000, 'xtol': 1e-16, 'gtol': 1e-16})
#result = sp.optimize.differential_evolution(objective_function, bounds, disp = True, popsize=300, tol = 1e-16, maxiter = 1000)
#result = sp.optimize.root(pump_equations, x0, method='trust-constr', bounds=bounds, options={'xtol': 1e-10, 'maxiter': 2000})

# Perform global optimization using basinhopping
# Basin Hopping Methods: Powell
result = sp.optimize.basinhopping(objective_function,x0=x0,minimizer_kwargs={"method": "Powell", "bounds": bounds},niter=1000,T=0.5,stepsize=1,disp=True)

basinhopping step 0: f 0.0247199
basinhopping step 1: f 2.17817e-07 trial_f 2.17817e-07 accepted 1  lowest_f 2.17817e-07
found new global minimum on step 1 with function value 2.17817e-07
basinhopping step 2: f 5.17608e-08 trial_f 5.17608e-08 accepted 1  lowest_f 5.17608e-08
found new global minimum on step 2 with function value 5.17608e-08
basinhopping step 3: f 7.09065e-08 trial_f 7.09065e-08 accepted 1  lowest_f 5.17608e-08
basinhopping step 4: f 3.07257e-08 trial_f 3.07257e-08 accepted 1  lowest_f 3.07257e-08
found new global minimum on step 4 with function value 3.07257e-08
basinhopping step 5: f 4.84738e-10 trial_f 4.84738e-10 accepted 1  lowest_f 4.84738e-10
found new global minimum on step 5 with function value 4.84738e-10
basinhopping step 6: f 1.83343e-05 trial_f 1.83343e-05 accepted 1  lowest_f 4.84738e-10
basinhopping step 7: f 7.32885e-09 trial_f 7.32885e-09 accepted 1  lowest_f 4.84738e-10
basinhopping step 8: f 5.65681e-10 trial_f 5.65681e-10 accepted 1  lowest_f 4.84738

C:\Users\Krai\AppData\Local\Temp\ipykernel_4804\3229465052.py:23: RuntimeWarning: invalid value encountered in sqrt
  F1 = (N1 * np.sqrt(M1/rho1) / (g * P1/(rho1*g))**(3/4)) - (N2 * np.sqrt(M2/rho2) / (g * P2/(rho2*g))**(3/4))


basinhopping step 48: f 5.51768e-10 trial_f 5.51768e-10 accepted 1  lowest_f 4.76879e-10
basinhopping step 49: f 4.91774e-10 trial_f 4.91774e-10 accepted 1  lowest_f 4.76879e-10
adaptive stepsize: acceptance rate 0.980000 target 0.500000 new stepsize 1.11111 old stepsize 1
basinhopping step 50: f 7.40524e-08 trial_f 7.40524e-08 accepted 1  lowest_f 4.76879e-10
basinhopping step 51: f 4.96465e-10 trial_f 4.96465e-10 accepted 1  lowest_f 4.76879e-10
basinhopping step 52: f 5.45798e-10 trial_f 5.45798e-10 accepted 1  lowest_f 4.76879e-10
basinhopping step 53: f 5.36572e-10 trial_f 5.36572e-10 accepted 1  lowest_f 4.76879e-10
basinhopping step 54: f 4.46983e-06 trial_f 4.46983e-06 accepted 1  lowest_f 4.76879e-10
basinhopping step 55: f 6.47468e-20 trial_f 6.47468e-20 accepted 1  lowest_f 6.47468e-20
found new global minimum on step 55 with function value 6.47468e-20
basinhopping step 56: f 2.24126e-08 trial_f 2.24126e-08 accepted 1  lowest_f 6.47468e-20
basinhopping step 57: f 2.93398e-09

In [11]:
mu2, rho2, M2, P2 = result.x[0], result.x[1], result.x[2], result.x[3]
Q2 = M2/rho2
specific_speed_1 = N1*np.sqrt(M1/rho1)/(P1/(rho1*g))**(3/4)
specific_speed_2 = N2*np.sqrt(M2/rho2)/(P2/(rho2*g))**(3/4)
print(f"Ns = {specific_speed_2:.4f}, Matching = {100-abs(specific_speed_1-specific_speed_2)/specific_speed_1*100} %")

flow_coeff_1 = Q1/N1/D1**3
flow_coeff_2 = Q2/N2/D2**3
print(f"Flow Coefficient Prototype = {flow_coeff_1}, Model = {flow_coeff_2}, Matching = {100-abs(flow_coeff_1-flow_coeff_2)/flow_coeff_1*100} %")

head_coeff_1 = g*P1/(rho1*g)/N1**2/D1**2
head_coeff_2 = g*P2/(rho2*g)/N2**2/D2**2
print(f"Head Coefficient Prototype = {head_coeff_1}, Model = {head_coeff_2}, Matching = {100-abs(head_coeff_1-head_coeff_2)/head_coeff_1*100} %")

Reysnold_1 = rho1*N1*D1**2/mu1
Reysnold_2 = rho2*N2*D2**2/mu2
print(f"Reynolds Number Prototype = {Reysnold_1:.4f}, Model = {Reysnold_2:.4f}, Matching = {100-abs(Reysnold_2-Reysnold_2)/Reysnold_1*100} %")

print(f"\nModel's Shaft Speed is {N2:.2f} RPM\n"
      f"Model's Impeller Diameter is {D2*1e3:.2f} mm\n"
      f"Model's Mass Flow Rate is {M2:.2f} Kg/s\n"
      f"Model's Pressure Across Pump is {P2/1e5:.2f} Bar\n"
      f"Model's Pressure Head is {P2/rho1/9.807:.2f} m HAN+H2O2\n"
      f"Model's Pressure Head is {P2/rho2/9.807:.2f} m Testing Fluid\n"
      f"Model's Pressure Head is {P2/1000/9.807:.2f} m Water\n"
      f"Model's Volumetric Flow Rate is {Q2*1000:.3f} L/s\n"
      f"Model's Fluid's Viscosity is {mu2*1000:.3f} mPa-s\n"
      f"Model's Fluid's Density is {rho2:.3f} kg/m^3\n"
      f"Model's Hydraulic Power = {Q2*P2:.3f} W\n"
      f"Model's Shaft Power= {Q2*P2/0.28:.3f} W\n"
      f"Model's Motor Torque = {Q2*P2/0.28/(N2*2*math.pi/60):.3f} N-m\n")

Ns = 17.7850, Matching = 99.99999999517006 %
Flow Coefficient Prototype = 0.0002262830139178427, Model = 0.00022628301386340289, Matching = 99.9999999759417 %
Head Coefficient Prototype = 0.0007844477815818548, Model = 0.000784447781506556, Matching = 99.99999999040105 %
Reynolds Number Prototype = 1133139.6554, Model = 1133139.6554, Matching = 100.0 %

Model's Shaft Speed is 2900.00 RPM
Model's Impeller Diameter is 80.00 mm
Model's Mass Flow Rate is 0.40 Kg/s
Model's Pressure Across Pump is 0.50 Bar
Model's Pressure Head is 3.24 m HAN+H2O2
Model's Pressure Head is 4.31 m Testing Fluid
Model's Pressure Head is 5.10 m Water
Model's Volumetric Flow Rate is 0.336 L/s
Model's Fluid's Viscosity is 19.398 mPa-s
Model's Fluid's Density is 1184.281 kg/m^3
Model's Hydraulic Power = 16.800 W
Model's Shaft Power= 60.001 W
Model's Motor Torque = 0.198 N-m

